## Use PhariaSearch with PhariaKernel Skills

### Load the Enviroment Variables

In [ ]:
from validation_utils import validate_environment

# Run environment validation
validate_environment()

# If validation fails, DO NOT proceed with the rest of the tutorial
# Fix the issues identified above first!

## Use PhariaSearch with PhariaKernel Skills

### Load Environment Variables

Now we'll load all the configuration parameters from your `.env` file. These include API endpoints, authentication tokens, and the names of resources we'll use for the PhariaKernel skill.

In [ ]:
from os import getenv
from dotenv import load_dotenv

load_dotenv(override=True)

PHARIA_AI_TOKEN = getenv("PHARIA_AI_TOKEN")
PHARIA_KERNEL_ADDRESS = getenv("PHARIA_KERNEL_ADDRESS")
NAMESPACE = getenv("PHARIA_DATA_NAMESPACE")
COLLECTION = getenv("PHARIA_DATA_COLLECTION")
INDEX = getenv("INDEX")

print("\n")
print("\033[92m✅ SUCCESS: Environment variables loaded\033[0m")
print(f"   • Namespace: {NAMESPACE}")
print(f"   • Collection: {COLLECTION}")
print(f"   • Index: {INDEX}")
print(f"   • Kernel Address: {PHARIA_KERNEL_ADDRESS}")
print(f"   • Token: {'✓ Set' if PHARIA_AI_TOKEN else '✗ Missing'}")
print("\n")

### Define the Skill

PhariaSearch is natively integrated within PhariaKernel to reduce the effort necessary to develop a RAG (Retrieval-Augmented Generation) based system.

The PhariaSkill SDK is available on PyPI (https://pypi.org/project/pharia-skill/) and can be installed with `pip install pharia-skill`. Let's start by importing the necessary dependencies to build our RAG skill.

<div class="alert alert-info">
PhariaKernel supports only semantic search and not filter indexes or hybrid indexes. This will be supported in the near future.
</div>

In [ ]:

from pharia_skill import ChatParams, Csi, IndexPath, Message, skill
from pydantic import BaseModel

print("\n")
print("\033[92m✅ SUCCESS: Dependencies imported\033[0m")
print("   • PhariaSkill SDK components loaded")
print("   • Pydantic for data validation")
print("\n")

#### Define Input and Output Models

Let's define the basic Input and Output structure to make our RAG skill work. These Pydantic models ensure type safety and validation for our skill's interface.

In [ ]:
class Input(BaseModel):
    question: str
    namespace: str = NAMESPACE
    collection: str = COLLECTION
    index: str = INDEX

class Output(BaseModel):
    answer: str | None

print("\n")
print("\033[92m✅ SUCCESS: Data models defined\033[0m")
print("   • Input model: question + search parameters")
print("   • Output model: answer (nullable)")
print("   • Default values from environment variables")
print("\n")

#### Implement RAG Skill Logic

Finally, let's define our AI logic within the skill. This function will:
1. Search for relevant documents using PhariaSearch
2. Use the found documents as context for the LLM
3. Generate an answer using the context and question

In [ ]:

@skill
def rag_skill(csi: Csi, input: Input) -> Output:
    index = IndexPath(
        namespace=input.namespace,
        collection=input.collection,
        index=input.index,
    )

    if not (documents := csi.search(index, input.question, 3, 0.5)):
        return Output(answer=None)

    context = "\n".join([d.content for d in documents])
    content = f"""Using the provided context documents below, answer the following question accurately and comprehensively. If the information is directly available in the context documents, cite it clearly. If not, use your knowledge to fill in the gaps while ensuring that the response is consistent with the given information. Do not fabricate facts or make assumptions beyond what the context or your knowledge base provides. Ensure that the response is structured, concise, and tailored to the specific question being asked.

Input: {context}

Question: {input.question}
"""
    message = Message.user(content)
    params = ChatParams(max_tokens=512)
    response = csi.chat("llama-3.1-8b-instruct", [message], params)
    return Output(answer=response.message.content)

print("\n")
print("\033[92m✅ SUCCESS: PhariaSearch RAG skill function defined\033[0m")
print("   • Function: rag_skill() - retrieves documents and generates answers")
print("   • Search: Retrieves top 3 documents with similarity > 0.5")
print("   • Context: Combines retrieved documents for LLM context")
print("   • Generation: Uses llama-3.1-8b-instruct with 512 token limit")
print("   • Returns: Structured answer or None if no documents found")
print("\n")

### Test Your Skill with DevCSI

Now that the Skill is defined, it is possible to test it locally using the DevCSI provided by PhariaKernel. This creates a development environment that connects to your PhariaSearch collection and allows you to test the RAG functionality.

The DevCSI requires the environment variables we loaded earlier:
- `PHARIA_AI_TOKEN` - Authentication token for API access
- `PHARIA_KERNEL_ADDRESS` - PhariaKernel service endpoint

In [ ]:
from pharia_skill.testing import DevCsi

# Initialize DevCSI for testing
csi = DevCsi()

print("\033[92m✅ SUCCESS: DevCSI initialized\033[0m")
print("\n")

# Test the skill with a sample question
question = "What is the influence of Robert Moses?"
result = rag_skill(csi, Input(question=question))

# Display results with beautiful formatting
print("\033[96m" + "="*60 + "\033[0m")
print(f"\033[96m📋 QUESTION:\033[0m {question}")
print("\033[96m" + "="*60 + "\033[0m")
print()

if result.answer:
    print("\033[92m🤖 ANSWER:\033[0m")
    print(result.answer)
    print()
    print("\033[92m✅ SUCCESS: Skill execution completed\033[0m")
else:
    print("\033[93m⚠️  WARNING: No relevant documents found\033[0m")

print("\033[96m" + "="*60 + "\033[0m")

### Move the skill to production

Once the Skill is ready, it is possible to deploy it to production by following the steps in the <a href="https://docs.aleph-alpha.com/products/pharia-ai/pharia-studio/pharia-kernel/quick_start">Quick Start with Kernel skills</a>